# Yolo Image Object Detection

In [1]:
import cv2,os,scipy
import warnings
import numpy as np
from matplotlib.pyplot import imshow
warnings.filterwarnings('ignore')

In [2]:
from yolo_utils import read_classes, read_anchors, generate_colors,draw_boxes,preprocess_image
from yad2k.models.keras_yolo import yolo_head
from yolo_algo_computation import yolo_eval
import tensorflow as tf
from keras import backend as K
from keras.models import load_model

Using TensorFlow backend.


# Initialize the session and yolo models

In [3]:
sess = K.get_session()
class_names = read_classes("model_data/coco_classes.txt")
anchors = read_anchors("model_data/yolo_anchors.txt")
image_shape = (720., 1280.)    
yolo_model = load_model("model_data/yolo.h5")

In [4]:
yolo_outputs = yolo_head(yolo_model.output, anchors, len(class_names))
scores, boxes, classes,boxes_before_nms = yolo_eval(yolo_outputs, image_shape)

# Predict on images and transform the bounding boxes

In [5]:
def predict(image_file):
    image, image_data = preprocess_image("images_input/" + image_file, model_image_size = (608, 608))
    out_scores, out_boxes, out_classes,boxes_before_nms_out= sess.run([scores,boxes,classes,boxes_before_nms],
        feed_dict={yolo_model.input:image_data, K.learning_phase(): 0}
    )
    final_boxes = (out_scores, out_boxes, out_classes)
    return final_boxes,boxes_before_nms_out,image

# Draw bounding boxes on images

In [6]:
def draw_bounding_box(boxes,image,image_file,dir_name):
    
    colors = generate_colors(class_names)
    out_scores, out_boxes, out_classes = boxes
    
    draw_boxes(image, out_scores, out_boxes, out_classes, class_names, colors)
    image.save(os.path.join(dir_name, image_file), quality=90)
    output_image = scipy.misc.imread(os.path.join(dir_name, image_file))

# Iterate the images_input directory and perform prediction

In [10]:
for dirpath, dirnames, filenames in os.walk('images_input'):
    print(filenames)

['car_person.jpg', 'market.jpg']


In [11]:
for img in filenames:
    final_boxes,boxes_before_nms_out,image = predict(img)
    draw_bounding_box(final_boxes,image,img,'image_files_output')
    draw_bounding_box(boxes_before_nms_out,image,img,'image_files_output_before_nms')